In [19]:
import pandas as pd # type: ignore
import numpy as np # type: ignore
from typing import Union, List
from sklearn.preprocessing import StandardScaler, MinMaxScaler # type: ignore
from sklearn.decomposition import PCA                   # type: ignore

import warnings

warnings.filterwarnings(action='ignore')

In [20]:
class Preprocessor:
    def __init__(self):
        self.file_path = ""
        self.folder_path = ""
        self.df = pd.DataFrame()
    
    def __init__(self, data_file_path:str="", folder_path:str=""):
        self.file_path = data_file_path
        self.folder_path = folder_path
        self.df = pd.DataFrame()
        
    def load_origin_file(self, file_path:str)->None:
        if file_path!="":
            self.file_path = file_path
        self.df = pd.read_csv(self.file_path)
        
    def drop_columns(self, drop_columns_file_path:str = "drop_columns_0411.txt")->None:
        with open(drop_columns_file_path, mode='r') as f:
            drop_fields = f.readlines()
            drop_fields = [drop_field.strip('\n') for drop_field in drop_fields]
        self.df.drop(columns=drop_fields, inplace=True)
    
    def __preprocess_target_variable(self, target_variable:str="loan_status")->None:
        # loan_status가 "current", "issued", "policy" 인 행을 필터링하여 삭제
        modified_df = self.df[~self.df[target_variable].isin(['Current', 'Issued', 'Does not meet the credit policy. Status:Fully Paid', 'Does not meet the credit policy. Status:Charged Off'])]
        # risk = 1, safe = 0 으로 처리
        modified_df.loc[modified_df['loan_status'].isin(['Fully Paid', 'In Grace Period']), 'loan_status'] = 0
        modified_df.loc[modified_df['loan_status'].isin(['Charged Off', 'Default', 'Late (16-30 days)', 'Late (31-120 days)']), 'loan_status'] = 1
        modified_df['loan_status'] = modified_df['loan_status'].astype('int')
        self.df = modified_df
        
    ## 5. 데이터 처리용 함수
    def __delete_suffix(self, term:str)->int:
        '''첫 단어만을 저장하는 함수'''
        term = term.strip().split()[0]
        return int(term)

    def __delete_suffix_percentage(self, term:str)->float:
        '''%를 자르는 함수'''
        term = term.strip('%')
        return float(term)
    
    def __fill_na_with_value(self, columns:List[str], filling_value:Union[str, int])->None:
        '''
        df: dataframe to fill NA
        column_name : column name to change NA values
        filling_value : value type or just value to fill column's NA
        '''
        for column_name in columns:
            if filling_value=="mode":
                mode_value = self.df[column_name].mode()[0]
            elif filling_value=="median":
                mode_value = self.df[column_name].median()
            else:
                mode_value = filling_value
            self.df[column_name].fillna(mode_value, inplace=True)
        
    def __preprocessing_na(self, is_train:bool)->None:
        '''
        'acc_open_past_24mths', 확인필요
        'avg_cur_bal', 확인필요
        '''
        ## 결측 처리
        # 결측 개수가 1천 건 이하인 경우는 해당 데이터(row) 삭제
        self.df.dropna(subset=['chargeoff_within_12_mths','collections_12_mths_ex_med','dti',
                                                'pub_rec_bankruptcies','revol_util','tax_liens'], inplace=True)
        
        if not is_train:
            # A1. 최빈값 대체
            self.__fill_na_with_value(columns=['mo_sin_old_il_acct', 'mths_since_recent_bc', 'mths_since_recent_inq', 'emp_length'], filling_value='mode')
            
            # A2. 중앙값 대체
            self.__fill_na_with_value(columns=['bc_open_to_buy'], filling_value='median')
        # B. 2015년 대체
        # is_after_2015 컬럼 생성. all_util 변수를 기준으로 사용
        self.df['is_after_2015'] = self.df['all_util'].apply(lambda x: 0 if pd.isnull(x) else 1)
        # 결측값을 0으로 채우기
        
        # C. 2012년 대체
        # is_after_2012 컬럼 생성. pct_tl_nvr_dlq 변수를 기준으로 사용
        self.df['is_after_2012'] = self.df['pct_tl_nvr_dlq'].apply(lambda x: 0 if pd.isnull(x) else 1)
        # D. 결측 0 대체
        if not is_train:
            self.__fill_na_with_value(columns=['annual_inc_joint','dti_joint','revol_bal_joint', 'open_acc_6m',
                                           'open_act_il', 'open_il_12m', 'open_il_24m', 'total_bal_il',
                                           'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'total_cu_tl', 'mths_since_rcnt_il',
                                           'tot_cur_bal', 'total_rev_hi_lim', 'mo_sin_old_rev_tl_op',
                                           'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc', 'num_bc_sats', 'num_bc_tl',
                                           'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl', 'num_il_tl',
                                           'num_op_rev_tl','num_rev_accts','num_rev_tl_bal_gt_0','num_sats','num_tl_120dpd_2m','num_tl_30dpd',
                                           'num_tl_90g_dpd_24m','num_tl_op_past_12m','pct_tl_nvr_dlq','tot_hi_cred_lim','total_bal_ex_mort',
                                           'total_bc_limit','total_il_high_credit_limit'], filling_value=0)
        
        
    def __convert_object_to_numeric(self, column_name:str)->pd.DataFrame:
        unique_values = sorted(self.df[column_name].unique())
        value_map = {value:index for index, value in enumerate(unique_values)}
        self.df[column_name] = self.df[column_name].apply(lambda x:value_map.get(x))
        return self.df
    
    def __convert_object_to_one_hot(self, column_name:str)->None:
        encoded = pd.get_dummies(self.df[column_name], drop_first=True, prefix='cat')
        self.df = pd.concat([self.df, encoded], axis=1)
        self.df.drop(column_name, axis=1, inplace=True)
        
    def __preprocessing_objects(self)->None:
        # term
        self.df['term'] = self.df['term'].apply(self.__delete_suffix)
        # emp_length
        self.df = self.df.dropna(subset=['emp_length'])
        self.df['emp_length'] = self.df['emp_length'].astype(str).apply(lambda x: x.replace(' years','').replace(' year','').replace('+','').replace('< 1', '0'))
        self.df['emp_length'] = self.df['emp_length'].astype(int)
        # revol_util
        self.df['revol_util'] = self.df['revol_util'].apply(self.__delete_suffix_percentage)
        self.df['int_rate'] = self.df['int_rate'].apply(self.__delete_suffix_percentage)
        ## numeric
        # application_type
        self.df = self.__convert_object_to_numeric('application_type')
        # sub_grade
        self.df = self.__convert_object_to_numeric('sub_grade')

        ## one-hot
        # home_ownership
        self.df['home_ownership'] = self.df['home_ownership'].replace(['ANY', 'OTHER', 'NONE'], 'OTHERS')
        self.__convert_object_to_one_hot('home_ownership')
        # self.__purpose()
        # purpose
        self.__convert_object_to_one_hot('purpose')
        self.__verification_status()

    def __Multicollinearity(self)->None:
        # self.df.drop(columns=['fico_range_low'], inplace=True)
        self.df['fico_avg'] = (self.df['fico_range_low'] + self.df['fico_range_high'])/2
        self.df.drop(columns=['fico_range_low', 'fico_range_high'], inplace=True)
    def __purpose(self):
        self.df['purpose'] = self.df['purpose'].map({"small_business":"essential_purpose", 
                                                     "house":"essential_purpose", 
                                                     "moving":"essential_purpose",
                                                     "medical":"essential_purpose",
                                                     "renewable_energy":"essential_purpose",
                                                     "other":"essential_purpose",
                                                     "wedding":"optional_purpose",
                                                     "vacation":"optional_purpose",
                                                     "car":"optional_purpose",
                                                     "home_improvement":"optional_purpose",
                                                     "educational":"optional_purpose",
                                                     "debt_consolidation":"debt_consolidation",
                                                     "credit_card":"credit_card",})
    def __verification_status(self):
        self.df['verification_status'] = self.df['verification_status'].map({"Not Verified":False, 'Source Verified':True, 'Verified':True})

    def __log_transform(self):
        variables = [
          "all_util", "annual_inc", "annual_inc_joint", "bc_open_to_buy", # "avg_cur_bal",
          "delinq_amnt", "dti", "max_bal_bc", "mo_sin_old_il_acct", "mo_sin_old_rev_tl_op",
          "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc", "mths_since_rcnt_il",
          "mths_since_recent_bc", "num_accts_ever_120_pd", "num_actv_bc_tl", "num_actv_rev_tl",
          "num_bc_sats", "num_bc_tl", "num_il_tl", "num_op_rev_tl", "num_rev_accts",
          "num_rev_tl_bal_gt_0", "num_sats", "open_acc", "open_acc_6m", "open_act_il",
          "open_il_12m", "open_il_24m", "open_rv_12m", "open_rv_24m", "pub_rec_bankruptcies",
          "revol_bal", "revol_bal_joint", "tax_liens", "tot_cur_bal", "tot_hi_cred_lim",
          "total_acc", "total_bal_ex_mort", "total_bal_il", "total_bc_limit", "total_cu_tl",
          "total_il_high_credit_limit", "total_rev_hi_lim"
        ]
        self.df = self.df.loc[~(self.df[variables]<0).any(axis=1)]
        # 각 변수에 대해 로그 변환 수행 및 기존 변수 삭제
        for var in variables:
            # 로그 변환 후 변수 이름에 '_log'를 추가하여 새로운 변수 생성
            new_var = var + "_log"
            # 해당 변수가 0보다 큰 경우에만 로그 변환 수행하여 음수 무한대를 방지
            # 로그 변환 후에는 기존 값이 0인 경우 음수 무한대로 처리되므로 이에 대한 처리도 필요
            # 여기서 로그는 자연로그 (밑이 e인 로그)
            self.df[new_var] = np.log(self.df[var] + 1)  # 0이 아닌 값이어야 하므로 +1 추가
            # 기존 변수 삭제
            self.df.drop(columns=[var], inplace=True)
        return self.df
    
    def preprocess(self, is_train:bool=True)->None:
        # loan_status 제외 모든 column이 결측치(na)인 행 제거 (1개 행 제거됨)
        self.df.dropna(subset=self.df.columns.difference(['loan_status']),how='all', inplace=True)
        self.__preprocess_target_variable()
        # 결측치 제거
        self.__preprocessing_na(is_train)
        ## object 처리하기
        self.__preprocessing_objects()
        ## 다중공선성 제거 - 0419 추가
        self.__Multicollinearity()
        ## 로그변환 - 0419 추가
        self.__log_transform()
        # index 재설정
        self.df.reset_index(drop=True, inplace=True)
        self.df.dropna(subset=self.df.columns.difference(['loan_status']),inplace=True)
        self.df.reset_index(drop=True, inplace=True)
        
    def do_scaling(self, scaler:StandardScaler|MinMaxScaler, fitted_scaler=None):
        all_features = self.df.columns
        nemeric_features = [feature for feature in all_features if 'cat' not in feature]
        numeric_df = self.df[nemeric_features].drop(columns=['loan_status'])
        if fitted_scaler==None:
            scaler = scaler()
            scaler.fit(numeric_df)
        else:
            scaler = fitted_scaler
        self.scaled_df = scaler.transform(numeric_df)
        return self.scaled_df, scaler
    
    def do_pca(self, threshold:float=0.95, fitted_pca=None, n_components=0):
        if fitted_pca==None:
            pca = PCA()
            pca.fit(self.scaled_df)
            explained_variance = pca.explained_variance_ratio_
            cumulative_explained_variance = np.cumsum(explained_variance)
            n_components = np.argmax(cumulative_explained_variance >= threshold) + 1
            column_names = [f'PC{i+1}' for i in range(n_components)]
            pca = PCA(n_components=n_components)
            pca.fit(self.scaled_df)
        else:
            pca = fitted_pca
            column_names = [f'PC{i+1}' for i in range(n_components)]
        pca_components = pca.transform(self.scaled_df).astype('float32')
        self.pca_df = pd.DataFrame(data=pca_components, columns=column_names)
        return pca, n_components
    
    def get_final_df(self):
        all_features = self.df.columns
        cat_features = [feature for feature in all_features if 'cat' in feature]
        cat_df = self.df[cat_features].reset_index(drop=True)
        target_df = self.df[['loan_status']].reset_index(drop=True)
        pca_df = self.pca_df.reset_index(drop=True)
        self.df = pd.concat([cat_df, pca_df, target_df], axis=1)
        return self.df
    
    def get_df(self)->pd.DataFrame:
        return self.df

In [22]:
train_preprocessor = Preprocessor()
# lending_club_2020_train.csv 파일이 있는 절대 경로 혹은 상대 경로를 명시해주세요
train_preprocessor.load_origin_file(file_path="data/lending_club_2020_train.csv")
# drop_columns_0410.txt 파일의 위치를 명시해주세요
train_preprocessor.drop_columns(drop_columns_file_path='texts/drop_columns_0411.txt')
# preprocess를 돌리면, addr_state를 제외한 object field 및, na(결측치) 처리됩니다.
train_preprocessor.preprocess(is_train=True)
train_df = train_preprocessor.get_df()
train_df['total_loan_amnt'] = train_df['loan_amnt']*train_df['int_rate']
# train_df['recovery_rate'] = (total_rec_prncp+total_rec_int)/(loan_amnt*(1+int_rate/100))
_, train_scaler = train_preprocessor.do_scaling(StandardScaler)
train_pca, train_n_components = train_preprocessor.do_pca()
pca_train_df = train_preprocessor.get_final_df()

In [23]:
pca_train_df.head()

,verification_status,application_type,cat_OTHERS,cat_OWN,cat_RENT,cat_credit_card,cat_debt_consolidation,cat_educational,cat_home_improvement,cat_house,...,PC29,PC30,PC31,PC32,PC33,PC34,PC35,PC36,PC37,loan_status
0,True,1,False,False,False,False,True,False,False,False,...,-0.318541,-0.841641,-0.842586,0.154576,-0.886086,-0.237734,0.483355,-0.218849,0.246910,0
1,True,1,False,True,False,False,True,False,False,False,...,0.377248,0.103233,-1.322784,-0.238591,-0.076571,-0.358260,-0.347312,0.409799,-0.266933,0
2,True,1,False,False,False,False,False,False,True,False,...,0.572435,0.208873,-0.019608,-1.029818,-0.027465,0.072785,0.216529,0.264002,-0.318242,0
3,False,1,False,False,True,False,False,False,False,False,...,-0.791596,-0.791170,-0.152939,-0.946270,-0.120305,-0.178754,-0.219152,0.448407,-0.343266,1
4,True,1,False,False,False,False,True,False,False,False,...,0.090147,0.028818,0.511519,0.188991,-0.326748,-0.402671,-0.643823,-0.071025,0.109675,0


In [27]:
train_df.head()

KeyError: 'total_rec_prncp'

In [25]:
pca_train_df.to_csv("0515/pca_train.csv", index=False)
train_df.to_csv("0515/train.csv", index=False)

In [26]:
test_preprocessor = Preprocessor()
# lending_club_2020_train.csv 파일이 있는 절대 경로 혹은 상대 경로를 명시해주세요
test_preprocessor.load_origin_file(file_path="data/lending_club_2020_test.csv")
# drop_columns_0410.txt 파일의 위치를 명시해주세요
test_preprocessor.drop_columns(drop_columns_file_path='texts/drop_columns_0411.txt')
# preprocess를 돌리면, addr_state를 제외한 object field 및, na(결측치) 처리됩니다.
test_preprocessor.preprocess(is_train=False)
test_df = train_preprocessor.get_df()
test_df['total_loan_amnt'] = test_df['loan_amnt']*test_df['int_rate']
test_preprocessor.do_scaling(scaler=StandardScaler, fitted_scaler=train_scaler)
test_preprocessor.do_pca(fitted_pca=train_pca, n_components=train_n_components)
pca_test_df = test_preprocessor.get_final_df()

KeyError: 'loan_amnt'

In [ ]:
test_df.to_csv("0515/test.csv", index=False)
pca_test_df.to_csv("0515/pca_test.csv", index=False)

In [ ]:
test_df.head()

,verification_status,application_type,cat_OTHERS,cat_OWN,cat_RENT,cat_credit_card,cat_debt_consolidation,cat_educational,cat_home_improvement,cat_house,...,PC29,PC30,PC31,PC32,PC33,PC34,PC35,PC36,PC37,loan_status
0,True,1,False,False,False,False,True,False,False,False,...,-0.300122,-0.842412,-0.859828,0.058389,-0.884582,-0.233392,0.465023,-0.202891,0.248289,0
1,True,1,False,True,False,False,True,False,False,False,...,0.364723,0.088509,-1.302277,-0.341483,-0.037283,-0.357957,-0.341528,0.398430,-0.287310,0
2,True,1,False,False,False,False,False,False,True,False,...,0.569441,0.206622,0.045739,-1.048268,0.033333,0.070993,0.226170,0.265620,-0.334660,0
3,False,1,False,False,True,False,False,False,False,False,...,-0.799333,-0.813404,-0.054911,-0.911515,-0.099855,-0.181720,-0.221674,0.432048,-0.325791,1
4,True,1,False,False,False,False,True,False,False,False,...,0.076347,0.018758,0.520269,0.263199,-0.378870,-0.399057,-0.661569,-0.079237,0.117197,0


In [ ]:
pca_test_df.head()

,verification_status,application_type,cat_OTHERS,cat_OWN,cat_RENT,cat_credit_card,cat_debt_consolidation,cat_educational,cat_home_improvement,cat_house,...,PC29,PC30,PC31,PC32,PC33,PC34,PC35,PC36,PC37,loan_status
0,True,0,False,False,False,False,True,False,False,False,...,9.737289,-0.268816,-2.884273,-9.819909,-4.266641,4.327185,-6.089932,-0.109370,-0.409178,0
1,False,0,False,False,False,False,True,False,False,False,...,9.077317,0.793978,-5.632853,-11.083260,-5.978025,2.636610,-4.517081,0.490578,-0.805114,0
2,True,0,False,False,True,True,False,False,False,False,...,10.030536,-0.945842,-2.780431,-8.954631,-4.544154,3.338429,-4.986498,-0.351795,-0.150110,1
3,False,0,False,False,False,False,False,False,False,False,...,9.797421,-0.025851,-4.820011,-11.414575,-5.780971,3.877001,-4.626157,0.944111,-1.307588,0
4,True,0,False,True,False,False,True,False,False,False,...,9.236403,-0.474840,-3.356656,-9.157453,-3.677489,2.975740,-6.765935,-0.086596,-0.276601,0


In [ ]:
train_df.shape, pca_train_df.shape, test_df.shape, pca_train_df.shape

((27224, 86), (27224, 56), (27224, 56), (27224, 56))